In [13]:
import cv2
from mtcnn.mtcnn import MTCNN
import face_recognition
import numpy as np
import csv
import os

In [14]:
def initialize_mtcnn():
    return MTCNN()

In [15]:
def create_cropped_faces_folder(folder_name="cropped_faces"):
    os.makedirs(folder_name, exist_ok=True)
    return folder_name

In [16]:
def load_and_encode_known_faces(labeled_images):
    known_face_encodings = []
    known_face_labels = []

    for label, image_path in labeled_images.items():
        image = face_recognition.load_image_file(image_path)
        encodings = face_recognition.face_encodings(image)
        if encodings:
            known_face_encodings.append(encodings[0])
            known_face_labels.append(label)

    return known_face_encodings, known_face_labels

In [17]:
def initialize_video_capture(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    return cap, frame_width, frame_height, fps

In [18]:
def initialize_video_writer(output_video_path, frame_width, frame_height, fps):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    return cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

In [19]:
def process_frame(frame, detector, known_face_encodings, known_face_labels, frame_count, cropped_faces_folder):
    dataset_entries = []
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detections = detector.detect_faces(frame_rgb)
    detected_individuals = set()

    for i, detection in enumerate(detections):
        x, y, width, height = detection['box']
        top, right, bottom, left = y, x + width, y + height, x

        face_crop = frame_rgb[top:bottom, left:right]
        crop_filename = f"{cropped_faces_folder}/face_frame{frame_count}_detection{i}.jpg"
        cv2.imwrite(crop_filename, cv2.cvtColor(face_crop, cv2.COLOR_RGB2BGR))

        face_encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])
        if face_encodings:
            face_encoding = face_encodings[0]
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

            if True in matches:
                best_match_index = np.argmin(face_distances)
                label = known_face_labels[best_match_index]
                probability = 1 - face_distances[best_match_index]
                detected_individuals.add(label)
            else:
                label = "Unknown"
                probability = 0.0
        else:
            label = "Unknown"
            probability = 0.0

        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} ({probability:.2f})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        dataset_entries.append({
            "Frame": frame_count,
            "Label": label,
            "Confidence": probability,
            "BoundingBox": [top, right, bottom, left],
            "CroppedFacePath": crop_filename
        })

    return frame, dataset_entries, detected_individuals


In [20]:
def save_dataset_to_csv(dataset, csv_file_path):
    with open(csv_file_path, mode='w', newline='') as csv_file:
        fieldnames = ["Frame", "Label", "Confidence", "BoundingBox", "CroppedFacePath"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(dataset)

In [21]:
def process_video(video_path, output_video_path, labeled_images, attendance_file_path):
    detector = initialize_mtcnn()
    cropped_faces_folder = create_cropped_faces_folder()
    known_face_encodings, known_face_labels = load_and_encode_known_faces(labeled_images)

    cap, frame_width, frame_height, fps = initialize_video_capture(video_path)
    out = initialize_video_writer(output_video_path, frame_width, frame_height, fps)

    dataset = []
    detected_names = set()
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        annotated_frame, frame_entries, frame_detected_individuals = process_frame(
            frame, detector, known_face_encodings, known_face_labels, frame_count, cropped_faces_folder
        )
        dataset.extend(frame_entries)
        detected_names.update(frame_detected_individuals)
        out.write(annotated_frame)
        frame_count += 1

    cap.release()
    out.release()

    csv_file_path = "face_recognition_video_dataset.csv"
    save_dataset_to_csv(dataset, csv_file_path)

    attendance_data = load_attendance_data(attendance_file_path)
    updated_attendance = update_attendance(detected_names, attendance_data)
    save_updated_attendance(updated_attendance, attendance_file_path)

    print(f"Processed video saved to {output_video_path}")
    print(f"Dataset saved to {csv_file_path}")
    print(f"Attendance report updated in {attendance_file_path}")


In [22]:
# def save_sample_frames_with_boxes(video_path, detector, known_face_encodings, known_face_labels, sample_output_folder, num_frames=10):
#     cap = cv2.VideoCapture(video_path)
#     os.makedirs(sample_output_folder, exist_ok=True)
#     frame_count = 0
#     saved_frames = 0

#     while cap.isOpened() and saved_frames < num_frames:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         detections = detector.detect_faces(frame_rgb)

#         # Annotate the frame with bounding boxes and labels
#         for detection in detections:
#             x, y, width, height = detection['box']
#             top, right, bottom, left = y, x + width, y + height, x
#             face_encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])
            
#             if face_encodings:
#                 face_encoding = face_encodings[0]
#                 matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
#                 face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

#                 if True in matches:
#                     best_match_index = np.argmin(face_distances)
#                     label = known_face_labels[best_match_index]
#                     probability = 1 - face_distances[best_match_index]
#                 else:
#                     label = "Unknown"
#                     probability = 0.0
#             else:
#                 label = "Unknown"
#                 probability = 0.0

#             # Draw bounding box and label
#             cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
#             cv2.putText(frame, f"{label} ({probability:.2f})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#         # Save the annotated frame
#         sample_frame_path = f"{sample_output_folder}/frame{frame_count}.jpg"
#         cv2.imwrite(sample_frame_path, frame)
#         saved_frames += 1
#         frame_count += 1

#     cap.release()
#     print(f"Saved {saved_frames} sample frames with boxes to {sample_output_folder}")


In [24]:
def load_attendance_data(csv_file_path):
    attendance_data = {}
    if os.path.exists(csv_file_path):
        with open(csv_file_path, mode='r') as csv_file:
            reader = csv.DictReader(csv_file)
            for row in reader:
                attendance_data[row["Name"]] = row["Status"]
    else:
        print(f"Attendance file '{csv_file_path}' not found. Starting with an empty record.")
    return attendance_data


In [25]:
def update_attendance(detected_names, attendance_data):
    for name in detected_names:
        attendance_data[name] = "Present"
    return attendance_data

In [26]:
def save_updated_attendance(attendance_data, csv_file_path):
    with open(csv_file_path, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Name", "Status"])
        for name, status in attendance_data.items():
            writer.writerow([name, status])
    print(f"Updated attendance saved to {csv_file_path}")


In [23]:
# def save_attendance_report(detected_names, attendance_file_path):
#     with open(attendance_file_path, mode='w', newline='') as csv_file:
#         writer = csv.writer(csv_file)
#         writer.writerow(["Name", "Status"])
#         for name in detected_names:
#             writer.writerow([name, "Present"])
#     print(f"Attendance report saved to {attendance_file_path}")


In [27]:
if __name__ == "__main__":
    labeled_images = {
        "...s"
    }
    
    video_path = "K:\\My Drive\\Project\\img\\Sample.MOV"
    output_video_path = "output_video.mp4"
    attendance_file_path = "attendance.csv"

    process_video(video_path, output_video_path, labeled_images, attendance_file_path)

8/8 [==============================] - 0s 4ms/step
Updated attendance saved to attendance.csv
Processed video saved to output_video.mp4
Dataset saved to face_recognition_video_dataset.csv
Attendance report updated in attendance.csv
